<h2>Manually inspect images, define ROIs with labels and extract nuclei numbers positive for a cell marker within each label (3D stack to 2D MIP)</h2>

In [24]:
from pathlib import Path
import czifile
import nd2
import tifffile
import napari
import os
import numpy as np
import pandas as pd
from utils import list_images, read_image, segment_nuclei_2d, segment_marker_positive_nuclei

<h3>Define the directory where your images are stored (.nd2 or .czi files)</h3>

In [25]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("./raw_data/test_data")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

images

['raw_data\\test_data\\HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi',
 'raw_data\\test_data\\HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi']

<h3>Open each image in the directory</h3>
You can do so by changing the number within the brackets below <code>image = images[0]</code>. By changing the <code>slicing factor</code> you lose resolution but speed up processing times (check the results).

In [26]:
# Explore a different image to crop (0 defines the first image in the directory)
image = images[1]

# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Generate maximum intensity projection and extract filename
img_mip, filename = read_image(image, slicing_factor)

# Show image in Napari to define ROI
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img_mip)

Image displayed: HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1
Original Array shape: (4, 12, 3798, 2877)
MIP Array shape: (4, 3798, 2877)


<Image layer 'img_mip' at 0x1a0f1c2f460>

<h3>Label your regions of interest in Napari and explore the signal of your marker of interest</h3>

Make sure to set <code>n edit dim = 3</code> so the label propagates across all channels. Name your regions of interest as i.e. <code>DG</code>, <code>CA1</code>, <code>CA3</code> or <code>HIPPO</code>. If you do not draw any ROI the entire image will be analyzed.

Fnally, select the <code>img_mip</code> layer and play with the contrast limit to later set a threshold above which cells will be considered positive for said marker.

<video controls>
  <source src="./assets/napari_labels.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

In [27]:
# Code snippet to analyze cropped regions (ROIs) defined by labels or the full image if no ROI is drawn in Napari

# Initialize empty list to store the label name and Numpy arrays so we can loop across the different ROIs
layer_names = []
layer_labels = []

if len(viewer.layers) == 1:

    # Extract the xy dimensions of the input image
    img_shape = viewer.layers[0].data.shape
    img_xy_dims = img_shape[-2:]

    # Create a label covering the entire image
    label = np.ones(img_xy_dims)

    # Add a name and the label to its corresponding list
    layer_names.append("full_image")
    layer_labels.append(label)

else:

    for layer in viewer.layers:

        # Extract the label names
        label_name = layer.name
        # Ignore img_mip since it is not a user defined label
        if label_name == "img_mip":
            pass
        else:
            # Store label names
            layer_names.append(label_name)
            # Get the label data as a NumPy array to mask the image
            label = layer.data 
            layer_labels.append(label)

# Print the defined ROIs that will be analyzed
print(f"The following labels will be analyzed: {layer_names}")


The following labels will be analyzed: ['full_image']


<h3>Save user-defined label ROIs as .tiff files</h3>

In [28]:
# Save user-defined ROIs in a ROI folder under directory_path/ROI as .tiff files
# Subfolders for each user-defined label region
# Store using the same filename as the input image to make things easier

for label_name, label_array in zip(layer_names, layer_labels):

    if label_name == "full_image":
        print("Full image analyzed, no need to store ROIs")
        pass

    else:

        # Perform maximum intensity projection (MIP) from the label stack
        label_mip = np.max(label_array, axis=0)

        # We will create a mask where label_mip is greater than or equal to 1
        mask = (label_mip >= 1).astype(np.uint8)

        # Create ROI directory if it does not exist
        try:
            os.makedirs(directory_path / "ROIs" / label_name)
        except FileExistsError:
            pass

        # Construct path to store
        roi_path = directory_path / "ROIs" / label_name / f"{filename}.tiff"

        # Save mask (binary image)
        tifffile.imwrite(roi_path, mask)

Full image analyzed, no need to store ROIs


<h3>Define your nuclei/marker stacks and your marker/erosion thresholds</h3>

Modify the values for <code>nuclei_channel</code>, <code>marker_channel</code>, <code>nuclei_channel_threshold</code> and <code>erosion_factor</code>

In [29]:
# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3
marker_channel = 1

# Define the intensity threshold above which a cell is considered positive for a marker
marker_channel_threshold = 40

# Define your nuclei diameter, it speeds up nuclei detection, if unknown leave it as None
cellpose_nuclei_diameter = None

# Define the amount of blur applied to nuclei
# Blurs the mip_nuclei image to even out high intensity foci within the nucleus, the higher the value blurriness increases
# High values help segment sparse nuclei (CA and CTX regions) but as a drawback it merges nuclei entities that are very close together (DG region)
gaussian_sigma = 0

# Sets the amount of erosion that is applied to areas where the marker+ signal colocalizes with nuclear signal
# The higher the value, the stricter the conditions to consider a nuclei as marker+
erosion_factor = 4

# Slice the nuclei and marker stack
nuclei_img = img_mip[nuclei_channel, :, :]
marker_img = img_mip[marker_channel, :, :]

<h3>Mask the input image with the user defined labels and extract data</h3>

In [30]:
# Create an empty list to store all stats extracted from each image
stats = []

for label_name, label_array in zip(layer_names, layer_labels):

    # Perform maximum intensity projection (MIP) from the label stack
    label_mip = np.max(label_array, axis=0)

    # We will create a mask where label_mip is greater than or equal to 1
    mask = label_mip >= 1

    # Apply the mask to nuclei_img and marker_img, setting all other pixels to 0
    masked_nuclei_img = np.where(mask, nuclei_img, 0)
    masked_marker_img = np.where(mask, marker_img, 0)
    viewer.add_image(masked_nuclei_img, name=f"{label_name}_nuclei")
    viewer.add_image(masked_marker_img, name=f"{label_name}_marker")

    # Segment nuclei and return labels
    nuclei_labels = segment_nuclei_2d(masked_nuclei_img, gaussian_sigma, cellpose_nuclei_diameter)
    viewer.add_labels(nuclei_labels, name=f"{label_name}_nuclei_labels")

    # Save nuclei labels as .tiff files to reuse them later
    # Create nuclei_predictions directory if it does not exist
    try:
        os.makedirs(directory_path / "nuclei_preds" / label_name)
    except FileExistsError:
        pass

    # Construct path to store
    nuclei_preds_path = directory_path / "nuclei_preds" / label_name / f"{filename}.tiff"

    # Save mask (binary image)
    tifffile.imwrite(nuclei_preds_path, nuclei_labels)

    # Select marker positive nuclei
    nuclei_and_marker, eroded_nuclei_and_marker, marker_mip, processed_region_labels = segment_marker_positive_nuclei (nuclei_labels, masked_marker_img, marker_channel_threshold, erosion_factor)
    viewer.add_image(nuclei_and_marker, name=f"{label_name}_marker_nuclei_coloc")
    viewer.add_image(eroded_nuclei_and_marker, name=f"{label_name}_marker_nuclei_eroded")
    viewer.add_labels(processed_region_labels, name=f"{label_name}_marker+_nuclei")

    # Extract your information of interest
    total_nuclei = len(np.unique(nuclei_labels)) - 1
    marker_pos_nuclei = len(np.unique(processed_region_labels)) - 1

    # Calculate "%_marker+_cells" and avoid division by zero errors
    try:
        perc_marker_pos_cells = (marker_pos_nuclei * 100) / total_nuclei
    except ZeroDivisionError:
        perc_marker_pos_cells = 0

    # Create a dictionary containing all extracted info per masked image
    stats_dict = {
                "filename": filename,
                "ROI": label_name,
                "total_nuclei": total_nuclei,
                "marker+_nuclei": marker_pos_nuclei,
                "%_marker+_cells": perc_marker_pos_cells,
                "nuclei_ch": nuclei_channel,
                "marker_ch": marker_channel,
                "marker_int_threshold": marker_channel_threshold,
                "erosion_factor": erosion_factor,
                "cellpose_nuclei_diameter": cellpose_nuclei_diameter,
                "gaussian_sigma": gaussian_sigma,
                "slicing_factor": slicing_factor
                }

    # Append the current data point to the stats_list
    stats.append(stats_dict)    

<h3>Data saving</h3>


In [31]:
# Define output folder for results
results_folder = "./results/"

# Create the necessary folder structure if it does not exist
try:
    os.mkdir(str(results_folder))
    print(f"Output folder created: {results_folder}")
except FileExistsError:
    print(f"Output folder already exists: {results_folder}")

# Transform into a dataframe to store it as .csv later
df = pd.DataFrame(stats)

# Define the .csv path
csv_path = "./results/marker_+_label_2D.csv"

# Append to the .csv with new data points each round
df.to_csv(csv_path, mode="a", index=True, header=not os.path.isfile(csv_path))

# Show the updated .csv 
csv_df = pd.read_csv(csv_path)

csv_df

Output folder already exists: ./results/


,Unnamed: 0,filename,ROI,total_nuclei,marker+_nuclei,%_marker+_cells,nuclei_ch,marker_ch,marker_int_threshold,erosion_factor,cellpose_nuclei_diameter,gaussian_sigma,slicing_factor
0,0,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,1331,1158,87.002254,3,1,40,4,NaN,0,NaN
1,1,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,DG,1747,1390,79.564969,3,1,40,4,NaN,0,NaN
2,0,HI 1 Ipsilateral Mouse 8 slide 6 Neun Red Ca...,DG,1740,977,56.149425,3,1,40,4,NaN,0,NaN
3,1,HI 1 Ipsilateral Mouse 8 slide 6 Neun Red Ca...,CA,984,629,63.922764,3,1,40,4,NaN,0,NaN
4,0,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,full_image,5054,2782,55.045509,3,1,40,4,NaN,0,NaN
5,0,HI 1 Ipsilateral Mouse 8 slide 6 Neun Red Ca...,full_image,5172,1908,36.890951,3,1,40,4,NaN,0,NaN
